In [1]:
# %% test cuda
import torch #type: ignore
torch.cuda.is_available()

True

In [1]:
# %% update data
import socket
import src.data as data
data.DataFetcher.main()
if socket.gethostname() == 'mengkjin-server':
    data.DataFetcher_sql.DataFetcher_sql.update_since()

Use device name: NVIDIA GeForce RTX 4090
Unpack Update Files
['/home/mengkjin/Workspace/SharedFolder/DB_updater.240402111348.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240403111906.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240403115028.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240405000856.tar']
Fri Apr  5 00:14:05 2024 : All Updates Done! Cost 5.75 Secs
Fri Apr  5 00:14:05 2024 : download since!
Connection and Factor preparation finished!
Fri Apr  5 00:14:05 2024 : sellside/haitong.hf_factors from 20240330 to 20240405, total 1 periods
Start sellside/haitong.hf_factors:20240330-20240405 
Done sellside/haitong.hf_factors:20240330-20240405 
Fri Apr  5 00:14:13 2024 : sellside/haitong.dl_factors from 20240330 to 20240405, total 1 periods
Start sellside/haitong.dl_factors:20240330-20240405 
Done sellside/haitong.dl_factors:20240330-20240405 
Fri Apr  5 00:15:44 2024 : sellside/dongfang.hfq_chars from 20240330 to 20240405, total 1 periods
Start sells

In [1]:
# %% test a specific model
from run_model import main
main(process = 0 , rawname = 1 , resume = 0)

Use device name: NVIDIA GeForce RTX 4090


24-04-08 22:31:13|MOD:run_model   |: Model Specifics:
24-04-08 22:31:13|MOD:run_model   |: Start Process [Load Data]!


--Process Queue : Data + Train + Test
--Model_name is set to patch_tst_day!
{'random_seed': None,
 'verbosity': 2,
 'precision': 'float',
 'batch_size': 10000,
 'model_name': 'patch_tst_day',
 'model_module': 'patch_tst',
 'model_num': 2,
 'model_data_type': 'day',
 'labels': ['std_lag1_10'],
 'beg_date': 20170103,
 'end_date': 99991231,
 'interval': 120,
 'input_step_day': 5,
 'test_step_day': 1,
 'MODEL_PARAM': {'hidden_dim': [32, 64],
                 'seqlens': [{'day': 30, '30m': 30, 'dms': 30}],
                 'seq_len': [30],
                 'dropout': [0.1],
                 'attn_dropout': [0.1],
                 'num_output': [1],
                 'patch_len': 5,
                 'stride': 1,
                 'shared_embedding': True,
                 'shared_head': True,
                 'revin': False,
                 'act_type': 'gelu',
                 'n_layers': 2,
                 'n_heads': 16,
                 'd_ff': 256},
 'train_params': {'dataloader': {'sampl

24-04-08 22:31:16|MOD:run_model   |: Finish Process [Load Data]! Cost 3.4Secs
24-04-08 22:31:16|MOD:run_model   |: Start Process [Train Model]!


Pre-Norming method of [day] : [endpoint_division(True) , history_standardize(True)]


patch_tst_day #0 @20170103 LoadData Cost    3.4Secs


score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


FirstBite Ep#  0 : loss  0.99903, train-0.00065, valid 0.00981, max 0.0098, best 0.0098, lr1.0e-07
FirstBite Ep#  5 : loss  0.93146, train 0.08918, valid 0.06200, max 0.0725, best 0.0725, lr3.8e-03
FirstBite Ep# 10 : loss  0.90459, train 0.11230, valid 0.09677, max 0.0992, best 0.0992, lr1.3e-03
FirstBite Ep# 15 : loss  0.88332, train 0.13555, valid 0.11017, max 0.1102, best 0.1102, lr6.3e-04
FirstBite Ep# 20 : loss  0.88411, train 0.13377, valid 0.11752, max 0.1175, best 0.1175, lr1.3e-03
FirstBite Ep# 25 : loss  0.86538, train 0.15495, valid 0.11525, max 0.1194, best 0.1194, lr1.6e-04
FirstBite Ep# 30 : loss  0.86139, train 0.15904, valid 0.11806, max 0.1200, best 0.1200, lr3.1e-04
FirstBite Ep# 35 : loss  0.85723, train 0.16344, valid 0.11805, max 0.1200, best 0.1200, lr2.3e-04
Reset learn rate and scheduler at the end of epoch 39 , effective at epoch 40
FirstBite Ep# 40 : loss  0.84964, train 0.17084, valid 0.11348, max 0.1200, best 0.1200, lr1.0e-07
FirstBite Ep# 45 : loss  0.8871

                        keys  num_calls    total_time   avg_time
5    model_train/epoch/train       1247  42593.628161  34.156879
7                 train/loss     478476  15497.830467   0.032390
9    model_train/epoch/valid       1247   5092.549536   4.083841
8             train/backward     478476   4956.845997   0.010360
11                valid/loss     119712   4617.191616   0.038569
6              train/forward     478476   1418.733445   0.002965
13                save_model       1277    858.956181   0.672636
14        model_train/status       1247    842.639054   0.675733
18       model_test/forecast         30    553.277533  18.442584
10             valid/forward     119712    283.932547   0.002372
19              test/forward      10542     51.856817   0.004919
1          model_train/start         30     51.156410   1.705214
12        model_train/assess       1247     17.049443   0.013672
17          model_test/start         30     13.995042   0.466501
20                 test/l

In [1]:
from src.data import PreProcess
PreProcess.main(False)
PreProcess.main(True)

Use device name: NVIDIA GeForce RTX 4090


24-03-31 09:52:11|MOD:data_preprocessing|: if_train is False , Data Processing start!
24-03-31 09:52:11|MOD:data_preprocessing|: 3 datas :['y', 'trade_day', 'trade_30m']


Sun Mar 31 09:52:11 2024 : y start ...
labels blocks reading ret10_lag DataBase's ...... cost 0.39 secs
labels blocks reading ret20_lag DataBase's ...... cost 0.33 secs
labels blocks merging ...... cost 0.05 secs
models blocks reading risk_exp DataBase's ...... cost 1.66 secs
models blocks merging ...... cost 0.17 secs
y blocks process ...... cost 0.88 secs
y blocks masking ...... cost 0.05 secs
y blocks saving  ...... cost 0.14 secs
y blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:14 2024 : y finished! Cost 3.67 Seconds
Sun Mar 31 09:52:14 2024 : trade_day start ...
trade blocks reading day DataBase's ...... cost 0.56 secs
trade_day blocks merging ...... cost 0.00 secs
trade_day blocks process ...... cost 0.03 secs
trade_day blocks masking ...... cost 0.05 secs
trade_day blocks saving  ...... cost 0.16 secs
trade_day blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:15 2024 : trade_day finished! Cost 0.81 Seconds
Sun Mar 31 09:52:15 2024 : trade_30m start ...
trade blocks r

24-03-31 09:52:21|MOD:data_preprocessing|: Data Processing Finished! Cost 10.02 Seconds


... cost 1.13 secs
trade_30m blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:21 2024 : trade_30m finished! Cost 5.19 Seconds
